In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
features = pd.read_csv('../data/raw/features.csv')
states_code = pd.read_csv('../catch_data/data/states.csv')

In [2]:
states_dict = {'ST0100000000000' : 'Alabama', 'ST0200000000000' : 'Alaska', 'ST0400000000000' : 'Arizona', 'ST0500000000000' : 'Arkansas', 'ST0600000000000' : 'California', 'ST0800000000000' : 'Colorado', 'ST0900000000000' : 'Connecticut', 'ST1000000000000' : 'Delaware', 'ST1100000000000' : 'District of Columbia', 'ST1200000000000' : 'Florida', 'ST1300000000000' : 'Georgia', 'ST1500000000000' : 'Hawaii', 'ST1600000000000' : 'Idaho', 'ST1700000000000' : 'Illinois', 'ST1800000000000' : 'Indiana', 'ST1900000000000' : 'Iowa', 'ST2000000000000' : 'Kansas', 'ST2100000000000' : 'Kentucky', 'ST2200000000000' : 'Louisiana', 'ST2300000000000' : 'Maine', 'ST2400000000000' : 'Maryland', 'ST2500000000000' : 'Massachusetts', 'ST2600000000000' : 'Michigan', 'ST2700000000000' : 'Minnesota', 'ST2800000000000' : 'Mississippi', 'ST2900000000000' : 'Missouri', 'ST3000000000000' : 'Montana', 'ST3100000000000' : 'Nebraska', 'ST3200000000000' : 'Nevada', 'ST3300000000000' : 'New Hampshire', 'ST3400000000000' : 'New Jersey', 'ST3500000000000' : 'New Mexico', 'ST3600000000000' : 'New York', 'ST3700000000000' : 'North Carolina', 'ST3800000000000' : 'North Dakota', 'ST3900000000000' : 'Ohio', 'ST4000000000000' : 'Oklahoma', 'ST4100000000000' : 'Oregon', 'ST4200000000000' : 'Pennsylvania', 'ST4400000000000' : 'Rhode Island', 'ST4500000000000' : 'South Carolina', 'ST4600000000000' : 'South Dakota', 'ST4700000000000' : 'Tennessee', 'ST4800000000000' : 'Texas', 'ST4900000000000' : 'Utah', 'ST5000000000000' : 'Vermont', 'ST5100000000000' : 'Virginia', 'ST5300000000000' : 'Washington', 'ST5400000000000' : 'West Virginia', 'ST5500000000000' : 'Wisconsin', 'ST5600000000000' : 'Wyoming', 'ST7200000000000' : 'Puerto Rico'}
seriesids = []
states = []
new_states_dict = {}
for key, value in states_dict.items():
    new_key = 'LA' + 'S' + key + '03'
    new_states_dict[new_key] = value
    seriesids.append(new_key)
    states.append(value)

In [4]:
api_key = 'bd259761a8aa4ad8a77810e12083dedf'
headers = {'Content-type': 'application/json'}
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

data1 = json.dumps({"seriesid": seriesids[0:25], 'registrationkey': api_key, "startyear":"2010", "endyear":"2012"})
p1 = requests.post(url=url, data=data1, headers=headers)
data1 = p1.json()

data2 = json.dumps({"seriesid": seriesids[25:], 'registrationkey': api_key, "startyear":"2010", "endyear":"2012"})
p2 = requests.post(url=url, data=data2, headers=headers)
data2 = p2.json()

In [5]:
for i, j in enumerate(data1['Results']['series']):
    if i == 0:
        df = pd.DataFrame(data1['Results']['series'][i]['data'])
        df.drop(['footnotes', 'period'], axis=1, inplace=True)
        df['month'] = pd.to_datetime(df['periodName'], format='%B').dt.month
        df.drop('periodName', axis=1, inplace=True)
        df['state'] = states[i]
    else:
        temp = pd.DataFrame(data1['Results']['series'][i]['data'])
        temp.drop(['footnotes', 'period'], axis=1, inplace=True)
        temp['month'] = pd.to_datetime(temp['periodName'], format='%B').dt.month
        temp.drop('periodName', axis=1, inplace=True)
        temp['state'] = states[i]
        df = pd.concat([df, temp])
        

for i, j in enumerate(data2['Results']['series']):
    temp = pd.DataFrame(data2['Results']['series'][i]['data'])
    temp.drop(['footnotes', 'period'], axis=1, inplace=True)
    temp['month'] = pd.to_datetime(temp['periodName'], format='%B').dt.month
    temp.drop('periodName', axis=1, inplace=True)
    i += 25
    temp['state'] = states[i]
    df = pd.concat([df, temp])

In [6]:
df['value'] = df['value'].astype(float)
df['date'] = df[['year', 'month']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m')
df = df.loc[df['date']>='2010-02']
df = df.loc[df['date']<='2013-07']
df.drop(['year', 'month'], axis=1, inplace=True)
df.sort_values('date', inplace=True)

/tmp/ipykernel_7727/1851118240.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['date'] = df[['year', 'month']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)


In [7]:
cols = []
for i in features.columns:
    cols.append(i.lower())
features.columns = cols

features['date'] = pd.to_datetime(features['date'])
features['date'].dt.strftime('%Y-%m')
features['year'] = features['date'].dt.year
features['month'] = features['date'].dt.month
features = features.groupby(['store', 'year', 'month']).agg({'unemployment':'first', 'date':'first'}).reset_index().drop(['year', 'month'], axis=1)
features['date'] = features['date'].dt.strftime('%Y-%m')
features.sort_values(['date', 'store'], inplace=True)

In [8]:
df_pivot = df.pivot_table(index='date', columns='state', values='value').reset_index()
df2 = features.merge(df_pivot, on='date', how='left')
proba_list = []
stores = dict()
for store in df2['store'].unique():
    temp = df2.loc[df2['store']==store].drop(['date', 'store'], axis=1).corr()['unemployment'][1:].sort_values(ascending=False)
    stores[store] = [temp.head(1).index[0], temp.head(1).values[0]]
    proba_list.append(temp.head(1).values[0])

In [9]:
stores

{1: ['Hawaii', 0.9627113784084476],
 2: ['California', 0.9941713651851788],
 3: ['Nevada', 0.961842822934712],
 4: ['Ohio', 0.9891523501175239],
 5: ['Nevada', 0.9799922624838774],
 6: ['California', 0.9868781967222878],
 7: ['Colorado', 0.9827527993646366],
 8: ['Nevada', 0.9729075998989525],
 9: ['Nevada', 0.9898088851565604],
 10: ['Arizona', 0.99448689913108],
 11: ['Nevada', 0.961842822934712],
 12: ['Nevada', 0.993471525229763],
 13: ['Arizona', 0.99348735283274],
 14: ['New Hampshire', 0.9549489754616137],
 15: ['New York', 0.8819213461165573],
 16: ['Oregon', 0.9680571998276669],
 17: ['Hawaii', 0.9496217149697896],
 18: ['Connecticut', 0.9678776762428611],
 19: ['New York', 0.8819213461165573],
 20: ['New Hampshire', 0.9727496529313483],
 21: ['California', 0.9941713651851788],
 22: ['South Dakota', 0.9626645501755198],
 23: ['Massachusetts', 0.9858929762841975],
 24: ['New York', 0.3094987535494979],
 25: ['New Hampshire', 0.9727496529313483],
 26: ['Oregon', 0.98338254226365

In [10]:
np.mean(proba_list)

0.9509396632000059

In [11]:
'0.9509396632000059'

'0.9509396632000059'

In [12]:
features['state'] = features['store'].map(stores)
features['state'] = features['state'].apply(lambda x: x[0])
features.drop(['unemployment', 'date'], axis=1, inplace=True)
features.to_csv('../data/raw/features_with_states.csv', index=True)

In [13]:
features.head()

,store,state
0,1,Hawaii
42,2,California
84,3,Nevada
126,4,Ohio
168,5,Nevada


In [14]:
states_code

,state,state_code,region
0,Alabama,AL,Sul
1,Alaska,AK,Oeste
2,Arizona,AZ,Oeste
3,Arkansas,AR,Sul
4,California,CA,Oeste
5,Colorado,CO,Oeste
6,Connecticut,CT,Nordeste
7,Delaware,DE,Nordeste
8,Florida,FL,Sul
9,Georgia,GA,Sul


In [15]:
features = features.merge(states_code, on='state', how='left')